In [ ]:

# This notebook generates LD for pre intervention and post intervention population. Ne is estimated from the calculated LD. 
# The two metrics assess change in population size during the llineup trial. 

#################################################################################################################################################################

# Load packages

import pickle
import allel
import numpy as np
import itertools
import random
from scipy import stats
import os
import malariagen_data
from re import *
import seaborn as sns
import pandas as pd
import re
import matplotlib.pyplot as plt

# ensure reproducibility 
random.seed(42)

# Write a function to calculate burrows' delta from a pair of loci
def burrows_delta(calls, allele_i, allele_j, unbiased = True):
	allele_num = calls.shape[1] * 2
	# This only works if the alleles can only be 0 or 1
	p = ((np.sum(calls[0]) * allele_i) + (allele_num - np.sum(calls[0])) * (1 - allele_i)) / allele_num
	q = ((np.sum(calls[1]) * allele_j) + (allele_num - np.sum(calls[1])) * (1 - allele_j)) / allele_num
	hom_i = 2*allele_i
	hom_j = 2*allele_j
	het = 1
	double_homozygotes = np.sum(np.logical_and(calls[0] == hom_i, calls[1] == hom_j))
	hom_i__het_j = np.sum(np.logical_and(calls[0] == hom_i, calls[1] == het))
	het_i__hom_j = np.sum(np.logical_and(calls[0] == het, calls[1] == hom_j))
	het__het = np.sum(np.logical_and(calls[0] == het, calls[1] == het))
	delta = (2*double_homozygotes +
	         hom_i__het_j +
	         het_i__hom_j +
	         het__het / 2
	        ) / calls.shape[1] - 2*p*q
	if unbiased:
		delta = delta*calls.shape[1] / (calls.shape[1] - 1)
	return(delta)

# Write a function to calculate r_hat_delta:
def Weir_r_hat_delta(calls, allele_i, allele_j):
	allele_num = calls.shape[1] * 2
	# This only works if the alleles can only be 0 or 1
	p = ((np.sum(calls[0]) * allele_i) + (allele_num - np.sum(calls[0])) * (1 - allele_i)) / allele_num
	q = ((np.sum(calls[1]) * allele_j) + (allele_num - np.sum(calls[1])) * (1 - allele_j)) / allele_num
	hom_i = 2*allele_i
	hom_j = 2*allele_j
	h_i = np.sum(calls[0] == hom_i) / calls.shape[1]
	h_j = np.sum(calls[1] == hom_j) / calls.shape[1]
	delta_hat = burrows_delta(calls, allele_i, allele_j)
	r_hat_delta = delta_hat / np.sqrt( (p*(1-p) + (h_i - p**2)) * (q*(1-q) + (h_j - q**2)) )
	return(r_hat_delta)

# For a given pair of loci, calculate the mean of the r values for all pairs of alleles
def mean_r_squared_delta(calls):
	# Remove samples with missing calls.
	missing_calls = np.any(calls < 0, 0)
	calls = calls[:, ~missing_calls]
	r_00 = Weir_r_hat_delta(calls, 0, 0) ** 2
	r_01 = Weir_r_hat_delta(calls, 0, 1) ** 2
	r_10 = Weir_r_hat_delta(calls, 1, 0) ** 2
	r_11 = Weir_r_hat_delta(calls, 1, 1) ** 2
	return(np.mean([r_00, r_01, r_10, r_11]))

# Calculate the expected r^2 due only to sampling (see Table 1 in Weir 1979), based on sample size S
def expected_r_squared(S):
	if S < 30:
		E = 0.0018 + 0.907/S + 4.44/(S**2)
	else:
		E = 1/S + 3.19/(S**2)
	return(E)

# For a set of genotype calls, calculate the mean r_squared, adjusted to the expectation. In case there are
# more locus permutations than are computationally feasible, we can set the number of allele pairs to randomly
# choose (N_subset). If N_subset = None, use all permutations
def overall_r_squared(call_set, N_subset = 100000, mindist = 1, positions = None):
	# If we want the minimum distance between markers to be greater than 1, then we need to know the positions of the
	# markers
	if mindist > 1 and positions is None:
		raise Exception('If mindist > 1, a list of marker positions is needed.')
	# What is the number of possible permutations?
	possible_permutations = int(call_set.shape[0] * (call_set.shape[0] - 1) / 2)
	# If the number of possible permutations is smaller than N_subset, or if N_subset is None, use all permutations
	if N_subset is None or N_subset >= possible_permutations:
		# Create an iterator for the permutations
		permutations = itertools.combinations(range(call_set.shape[0]), r = 2)
		# Set up the vector of r values
		r = np.array([])
		for p in permutations:
			if mindist > 1:
				this_distance = positions[p[1]] - positions[p[0]]
				if this_distance < mindist:
					continue
			these_calls = call_set[p, :]
			r = np.append(r, mean_r_squared_delta(these_calls))
	# Otherwise, use a random subset of permutations
	else :
		# the code below calculated all permutations and then chose a random subset of them. I think that for the
		# number of locus pairs that we are thinking of using, it will be a lot quicker to just sample them randomly
		# and kick out any repeats
#		# Create an iterator for the permutations
#		permutations = itertools.combinations(range(call_set.shape[0]), r = 2)
#		# Choose which permutations we will use
#		select_permutations = np.sort(random.sample(range(possible_permutations), N_subset))
#		# The islice function will skip a number of steps ahead in the iterator, so we need to change this array of
#		# choices into the number of steps to skip from the previous one
#		select_steps = select_permutations - np.concatenate([[0], select_permutations[:-1] + 1])
#		# Set up the vector of r values
#		r = np.array([])
#		for i, s in enumerate(select_steps):
#			p = next(itertools.islice(permutations, s, s+1))
#			if mindist > 1:
#				this_distance = positions[p[1] - positions[p[0]]]
#				if this_distance < mindist:
#					continue
#			these_calls = call_set[p, :]
#			r = np.append(r, mean_r_squared_delta(these_calls))
		# We choose pairs of loci at random. I'm going to be lazy here and make it so we never use the same locus
		# twice. This will cause problems if N_subset is ever larger than the call_set / 2, but I don't think that
		# this is realistically going to happen.
		gamete1 = np.array(random.sample(range(call_set.shape[0]), N_subset))
		unused = np.setdiff1d(np.array(range(call_set.shape[0])), gamete1)
		gamete2 = np.random.choice(unused, N_subset, replace = False)
		permutations = zip(gamete1, gamete2)
		r = np.array([])
		for p in permutations:
			if mindist > 1:
				this_distance = positions[p[1] - positions[p[0]]]
				if this_distance < mindist:
					continue
			these_calls = call_set[p, :]
			r = np.append(r, mean_r_squared_delta(these_calls))
	return(r)

# Write a function to calculate r_squared using variants from two different chromosomes.
# N_subset is a bit misleading here. If you set it to None, the function will take all pairwise combinations of SNPs
# between the two chromosomes. But if you set N_subset to a value, it will just take that many pairs. So setting
# N_subset to the number of SNPs is not the same as setting it to None.
def twochrom_r_squared(call_set1, call_set2, N_subset=None):
	if N_subset is None or N_subset > min([call_set1.shape[0], call_set2.shape[0]]):
		# Create an iterator for the permutations
		permutations = itertools.product(range(call_set1.shape[0]), range(call_set2.shape[0]))
		print('Using all pairwise combinations.')
	else:
		gamete1 = np.array(random.sample(range(call_set1.shape[0]), N_subset))
		gamete2 = np.array(random.sample(range(call_set2.shape[0]), N_subset))
		permutations = zip(gamete1, gamete2)
		print('Using a subset of loci with one pair per selected locus.')
	# Set up the vector of r values
#	r = np.array([mean_r_squared_delta(np.vstack([call_set1[p[0], :], call_set2[p[1], :]])) for p in permutations])
	r = np.array([])
	for p in permutations:
		these_calls = np.vstack([call_set1[p[0], :], call_set2[p[1], :]])
		r = np.append(r, mean_r_squared_delta(these_calls))
	return(r)

# Write a function to calculate effective population size from the r_squared values and sample size
def Weir_Ne(r_squared_vector, S, c = None):
	expected = expected_r_squared(S)
	
	observed = np.mean(r_squared_vector)
	adjusted = observed - expected
	# If no recombination rate was given, use the equations from Waples & Do 2008
	if c is None:
		if S < 30:
			Ne = (0.308 + np.sqrt(0.308**2 - 2.08 * adjusted)) / (2*adjusted)
		else:
			Ne = ((1/3) + np.sqrt((1/9) - 2.76 * adjusted)) / (2*adjusted)
	# Otherwise, use the equations from Appendix 1 of Hollenbeck et al 2016
	else:
		gamma = ((1-c)**2 + c**2)/(2*c*(2-c))
		Ne = gamma / adjusted
	if Ne < 0:
		return np.Inf
	else:
		return(Ne)

# Write a function that will take genotypes and calculate the r squared between the two chromosomes
def process_genotypes(genotypes_1, genotypes_2, sample_size = None, all_pairwise_thresh = 100000):
	if (genotypes_1.shape[1] != genotypes_2.shape[1]):
		raise Exception('The two genotypes should have the same number of samples. ')
	if sample_size is None:
		g_full_1 = genotypes_1[:,:]
		g_full_2 = genotypes_2[:,:]
	else:
		g_full_1 = genotypes_1[:, :sample_size]
		g_full_2 = genotypes_2[:, :sample_size]
	# remove singletons
	singleton_filter_1 = np.logical_and(np.sum(g_full_1, 1) > 1, np.sum(g_full_1, 1) < (2*g_full_1.shape[1] - 1))
	g_full_1 = g_full_1[singleton_filter_1, :]
	singleton_filter_2 = np.logical_and(np.sum(g_full_2, 1) > 1, np.sum(g_full_2, 1) < (2*g_full_2.shape[1] - 1))
	g_full_2 = g_full_2[singleton_filter_2, :]
	# Remove sites where every sample is heterozygous, since this throws off the calculations,
	# and also these are probably dodgy sites anyway.
	full_het_filter_1 = np.all(g_full_1 == 1, 1)
	g1 = g_full_1[~full_het_filter_1, :]
	full_het_filter_2 = np.all(g_full_2 == 1, 1)
	g2 = g_full_2[~full_het_filter_2, :]
	# now calculate rsquard
	num_loci = min([g1.shape[0], g2.shape[0]])
	if num_loci < all_pairwise_thresh:
		rs = twochrom_r_squared(g1, g2)
	else:
		rs = twochrom_r_squared(g1, g2, num_loci)
	return(g1, g2, rs)




In [ ]:

#load malariagen data
ag3 = malariagen_data.Ag3(pre = True)
sample_sets=["1288-VO-UG-DONNELLY-VMF00168","1288-VO-UG-DONNELLY-VMF00219"]
meta = ag3.sample_metadata(
    sample_sets=sample_sets, 
    sample_query = "aim_species == 'gambiae'"
)
# Remove decimal and numbers after it in the "partner_sample_id" column not present in our meta data
meta['partner_sample_id'] = meta['partner_sample_id'].str.split('.').str[0]

# Convert "partner_sample_id" column in meta to float64 to match llineup_meta dtype
meta['partner_sample_id'] = pd.to_numeric(meta['partner_sample_id'])

#load llineup trial metadata
llineup_meta = pd.read_csv('~/lstm_scratch/network_scratch/llineup/llineup-genomics/data/ento_geno_plasmo_data.csv',
                       index_col = 0,
                      ).query("species=='An. gambiae'")
llineup_meta= llineup_meta.drop_duplicates(subset=['wgs.sample.id'])
llineup_meta= llineup_meta.set_index('wgs.sample.id')
llineup_meta = llineup_meta.reindex(index=meta['partner_sample_id'])#match order in malariagen meta data
llineup_meta=llineup_meta.reset_index()
llineup_meta['sample_id'] =meta['sample_id']#create sample id to merge with ag3 metadata
llineup_meta['LLIN_actual']=llineup_meta['LLIN.actual']

#add column control phase for grouping samples to pre and post interventions
rnd_map = {1: 'pre', 5: 'post'}

llineup_meta['control_phase'] = llineup_meta['RND'].map(rnd_map).fillna('intermediate')
ag3.add_extra_metadata(llineup_meta)




In [ ]:
#Get genotype calls and exclude the 2Rb and 2La chromosomal inversions in  chromosome 2

# #2Rb inversion 2RL:18000000-32000000
# #2La inversion 2RL:81545105-104545105

#Get calls for mosquitoes collected on the final round of collections(Post-intervention). 

#If needed, you can split by net type (PBO/ Non PBO) and/or location (eastern/ western Uganda)
sample_query = "control_phase==['post']"

meta = ag3.sample_metadata(
    sample_sets=sample_sets, 
    sample_query=sample_query,
)

# Randomly select sample IDs from the filtered metadata
sample_subset = meta.sample(n=275, random_state=42).sample_id.to_list()

# Create a formatted string for the sample_query parameter
sample_query_subset = "sample_id in ({})".format(", ".join(f"'{id}'" for id in sample_subset))

#get calls from two chromosomes as we estimate inter-chromosome LD
calls_1 = ag3.biallelic_snp_calls(region=['2RL:1-17990000', '2RL:32010000-81535105', '2RL:104555105-110909430'],
                                  sample_sets=sample_sets,
                                  sample_query=sample_query_subset,
                                 )
calls_2 = ag3.biallelic_snp_calls(region='3RL',
                                  sample_sets=sample_sets,
                                  sample_query=sample_query_subset,
                                 )

callset_1_post = calls_1['call_genotype']
callset_2_post = calls_2['call_genotype']

if np.any(callset_1_post['sample_id'] != callset_2_post['sample_id']):
    raise Exception('Callsets 1 and 2 should contain the same samples')

callset_1_post



In [ ]:
#Get calls for mosquitoes collected on the baseline round of collections(Pre-intervention). 

sample_query = "control_phase==['pre']"

meta = ag3.sample_metadata(
    sample_sets=sample_sets, 
    sample_query=sample_query,
)

# Randomly select sample IDs from the filtered metadata
sample_subset = meta.sample(n=275, random_state=42).sample_id.to_list()

# Create a formatted string for the sample_query parameter
sample_query_subset = "sample_id in ({})".format(", ".join(f"'{id}'" for id in sample_subset))

calls_1 = ag3.biallelic_snp_calls(region=['2RL:1-17990000', '2RL:32010000-81535105', '2RL:104555105-110909430'],
                                  sample_sets=sample_sets,
                                  sample_query=sample_query_subset,
                                 )
calls_2 = ag3.biallelic_snp_calls(region='3RL',
                                  sample_sets=sample_sets,
                                  sample_query=sample_query_subset,
                                 )

callset_1_pre = calls_1['call_genotype']
callset_2_pre = calls_2['call_genotype']

if np.any(callset_1_pre['sample_id'] != callset_2_pre['sample_id']):
	raise Exception('Callsets 1 and 2 should contain the same samples')

callset_1_pre

In [ ]:

callset_1_post = callset_1_post.compute()
callset_2_post = callset_2_post.compute()

genotypes_1_post = np.sum(callset_1_post, 2)
genotypes_2_post= np.sum(callset_2_post, 2)

# Remove sites with missing calldata
missing_filter_1 = np.any(genotypes_1_post < 0, 1)
genotypes_1_post = genotypes_1_post[~missing_filter_1, :]
missing_filter_2 = np.any(genotypes_2_post< 0, 1)
genotypes_2_post= genotypes_2_post[~missing_filter_2, :]

# Remove non_segregating loci
segregating_filter_1 = np.logical_and(np.sum(genotypes_1_post, 1) > 0, np.sum(genotypes_1_post, 1) < (2*genotypes_1_post.shape[1]))
genotypes_1_post = genotypes_1_post[segregating_filter_1, :].compute()
segregating_filter_2 = np.logical_and(np.sum(genotypes_2_post, 1) > 0, np.sum(genotypes_2_post, 1) < (2*genotypes_2_post.shape[1]))
genotypes_2_post= genotypes_2_post[segregating_filter_2, :].compute()

g1_post, g2_post, rs_post = process_genotypes(genotypes_1_post, genotypes_2_post)

g2_post.shape

In [ ]:
# Save the results
with open('Ne_allpop_post.pickle', 'wb') as f :
	pickle.dump((genotypes_1_post, genotypes_2_post, g1_post, g2_post,rs_post), f)

# What is the variation in actual population size estimate?
W = Weir_Ne(rs_post, genotypes_2_post.shape[1], c = 0.5)
print(W)

In [ ]:
callset_1_pre = callset_1_pre.compute()
callset_2_pre = callset_2_pre.compute()

genotypes_1_pre = np.sum(callset_1_pre, 2)
genotypes_2_pre= np.sum(callset_2_pre, 2)

# Remove sites with missing calldata
missing_filter_1 = np.any(genotypes_1_pre < 0, 1)
genotypes_1_pre = genotypes_1_pre[~missing_filter_1, :]
missing_filter_2 = np.any(genotypes_2_pre< 0, 1)
genotypes_2_pre= genotypes_2_pre[~missing_filter_2, :]

# Remove non_segregating loci
segregating_filter_1 = np.logical_and(np.sum(genotypes_1_pre, 1) > 0, np.sum(genotypes_1_pre, 1) < (2*genotypes_1_pre.shape[1]))
genotypes_1_pre = genotypes_1_pre[segregating_filter_1, :].compute()
segregating_filter_2 = np.logical_and(np.sum(genotypes_2_pre, 1) > 0, np.sum(genotypes_2_pre, 1) < (2*genotypes_2_pre.shape[1]))
genotypes_2_pre= genotypes_2_pre[segregating_filter_2, :].compute()

g1_pre, g2_pre, rs_pre = process_genotypes(genotypes_1_pre, genotypes_2_pre)

g2_pre.shape

In [13]:
#Save the results
with open('Ne_allpop_pre.pickle', 'wb') as f :
	pickle.dump((genotypes_1_pre, genotypes_2_pre, g1_pre, g2_pre, rs_pre), f)

In [ ]:

# # Load data from the pickle file
# with open('Ne_allpop_pre.pickle', 'rb') as f:
#  genotypes_1_pre, genotypes_2_pre, g1_pre, g2_pre, rs_pre= pickle.load(f)
# with open('Ne_allpop_pre.pickle', 'rb') as c:
#  genotypes_1_post, genotypes_2_post, g1_post, g2_post, rs_post= pickle.load(c)



In [ ]:
# What is the variation in actual population size estimate?
W_pre = Weir_Ne(rs_pre , genotypes_2_post.shape[1], c = 0.5)
print(W_pre)
W_post = Weir_Ne(rs_post ,genotypes_2_post.shape[1], c = 0.5)
print(W_post)

In [ ]:

# skip this step if you use the genotypes generated by process_genotype. only use if load genotypes_*

# filter singletons and hets
def filter_genotypes(genotypes_1, genotypes_2, sample_size = None, all_pairwise_thresh = 20000):
	if (genotypes_1.shape[1] != genotypes_2.shape[1]):
		raise Exception('The two genotypes should have the same number of samples. ')
	if sample_size is None:
		g_full_1 = genotypes_1[:,:]
		g_full_2 = genotypes_2[:,:]
	else:
		g_full_1 = genotypes_1[:, :sample_size]
		g_full_2 = genotypes_2[:, :sample_size]
	# remove singletons
	singleton_filter_1 = np.logical_and(np.sum(g_full_1, 1) > 1, np.sum(g_full_1, 1) < (2*g_full_1.shape[1] - 1))
	g_full_1 = g_full_1[singleton_filter_1, :]
	singleton_filter_2 = np.logical_and(np.sum(g_full_2, 1) > 1, np.sum(g_full_2, 1) < (2*g_full_2.shape[1] - 1))
	g_full_2 = g_full_2[singleton_filter_2, :]
	# Remove sites where every sample is heterozygous, since this throws off the calculations,
	# and also these are probably dodgy sites anyway.
	full_het_filter_1 = np.all(g_full_1 == 1, 1)
	g1 = g_full_1[~full_het_filter_1, :]
	full_het_filter_2 = np.all(g_full_2 == 1, 1)
	g2 = g_full_2[~full_het_filter_2, :]
	
	return(g1, g2)



In [5]:
g1_pre, g2_pre = filter_genotypes(genotypes_1_pre, genotypes_2_pre)
g1_post, g2_post = filter_genotypes(genotypes_1_post, genotypes_2_post)

In [ ]:
# Extract R-squared values from rs_pre and rs_post if not an array
rs_pre_rsq = rs_pre#[-1]
rs_post_rsq = rs_post#[-1]
# Add a small value to all rsquared values
if np.min(rs_pre_rsq) == 0 or np.min(rs_post_rsq) == 0:
    small_value = 0.0000001  # or any small value you prefer
rs_pre_rsq += small_value
rs_post_rsq += small_value
# Take the logarithm of R-squared values
rs_pre_rsq_log = np.log10(rs_pre_rsq)
rs_post_rsq_log = np.log10(rs_post_rsq)

# Combine the log R-squared values with corresponding labels
data = pd.DataFrame({'R-squared (log)': np.concatenate([rs_pre_rsq_log, rs_post_rsq_log]),
                     'Group': ['rs_pre'] * len(rs_pre_rsq_log) + ['rs_post'] * len(rs_post_rsq_log)})

# Set seaborn style and font scale
sns.set(style="whitegrid", font_scale=1.5)

# Create a figure with specific dimensions and resolution
plt.figure(figsize=(8, 6), dpi=300)

# Define custom colors for each group
colors = ['#1f77b4', '#ff7f0e']  # Blue and orange colors

# Create a box plot using Seaborn with custom colors
ax = sns.boxplot(x='Group', y='R-squared (log)', data=data, linewidth=2, hue='Group', palette=colors, legend=False)

# Remove grid lines on the main plot area
ax.grid(False)

# Set axis labels and title
#plt.title('Comparison of R-squared Values (rs_pre vs rs_post)', fontsize=14)
plt.xlabel('Groups', fontsize=12)
plt.ylabel('Log R-squared', fontsize=12)

# Set tick label font sizes
ax.tick_params(axis='x', labelsize=12)
ax.tick_params(axis='y', labelsize=12)

# Set linewidth of boxplot elements
for box in ax.artists:
    box.set_linewidth(2)

# Save the plot 
plt.savefig('Ld_allpop.png', dpi=300, bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
# use logs squares to reduce skewness

# R-squared values extracted from rs_pre and rs_post tuples
rs_pre_rsq = rs_pre#[-1]
rs_post_rsq = rs_post#[-1]

# Check for negative values in rs_pre_rsq
if np.any(rs_pre_rsq < 0):
    print("There are negative values in rs_pre_rsq.")
else:
    print("There are no negative values in rs_pre_rsq.")

# Check for zero values in rs_pre_rsq
if np.any(rs_pre_rsq == 0):
    print("There are zero values in rs_pre_rsq.")
else:
    print("There are no zero values in rs_pre_rsq.")

# Check for negative values in rs_post_rsq
if np.any(rs_post_rsq < 0):
    print("There are negative values in rs_post_rsq.")
else:
    print("There are no negative values in rs_post_rsq.")

# Check for zero values in rs_post_rsq
if np.any(rs_post_rsq == 0):
    print("There are zero values in rs_post_rsq.")
else:
    print("There are no zero values in rs_post_rsq.")

# Calculate and print median of rs_pre_rsq
median_rs_pre = np.median(rs_pre_rsq)
print("Median of pre_rsq from Eastern Uganda:", median_rs_pre)

# Calculate and print median of rs_post_rsq
median_rs_post = np.median(rs_post_rsq)
print("Median of post_rsq from Eastern Uganda:", median_rs_post)
# Count the number of pairs with an r-squared value of 1
pre_pairs_with_rsq_of_1 = np.sum(rs_pre_rsq > 0.99)

print("Number of pre intervention pairs with an r-squared value of >0.99:", pre_pairs_with_rsq_of_1)

# Count the number of pairs with an r-squared value of 1
post_pairs_with_rsq_of_1 = np.sum(rs_post_rsq > 0.99)

print("Number of post intervention pairs with an r-squared value of >0.99:", post_pairs_with_rsq_of_1)


In [ ]:
stats.mannwhitneyu(rs_pre,rs_post)

In [6]:
# Now write a function to get confidence intervals by picking a subset of the SNPs
# and using a different subset each time
def Weir_Ne_replication(call_set1, call_set2, N_subset, c = 0.5, n_replicates = 1000, p = 0.95):
	if call_set1.shape[1] != call_set2.shape[1]:
		raise Exception('The two callsets need to have the same number of samples.')
	if N_subset > min([call_set1.shape[0], call_set2.shape[0]]):
		raise Exception('N_subset cannot be larger than the smaller of the two genotype matrices.')
	S = call_set1.shape[1]
	unsorted_Ws = np.array([])
	for i in range(n_replicates):
		print(i)
		rs_i = twochrom_r_squared(call_set1, call_set2, N_subset)
		unsorted_Ws = np.append(unsorted_Ws, Weir_Ne(rs_i, S, c))
	Ws = np.sort(unsorted_Ws)
	# Work out the indices of the percentiles. Because of floating point issues, we need to dance around a bit here.
	min_perc_index = n_replicates * round(1-p, 3) / 2
	max_perc_index = int(np.ceil(n_replicates - min_perc_index) - 1)
	min_perc_index = int(min_perc_index)
	# Now get the mean and confidence intervals
	W = np.mean(Ws)
	min_perc = Ws[min_perc_index]
	max_perc = Ws[max_perc_index]
	# Now report the mean and confidence intervals, and then all the boostrapped values
	return(W, min_perc, max_perc, Ws)

In [10]:

# Alternatively, we use a fixed subset of SNPs, and we boostrap over samples. As with
# the non-bootstrapping function we don't quite do the same thing if we set a subset
# and if we don't. If don't stipulate a subset, we use all pairwise combinations of
# loci between the two chromosomes. If we do, we just assign each locus on chrom 1 to
# a given locus on chrom 2.
def Weir_Ne_bootstrap_by_sample(call_set1, call_set2, c = 0.5, subsample_size = 247, N_subset = 100000, n_bootstrap = 1000, p = 0.95):
	if call_set1.shape[1] != call_set2.shape[1]:
		raise Exception('The two callsets need to have the same number of samples.')
	S = call_set1.shape[1]
	if N_subset is not None and N_subset <= min([call_set1.shape[0], call_set2.shape[0]]):
		# Reduce the call_sets to a subset of them
		subset1 = np.array(random.sample(range(call_set1.shape[0]), N_subset))
		call_set1 = call_set1[subset1, :]
		subset2 = np.array(random.sample(range(call_set2.shape[0]), N_subset))
		call_set2 = call_set2[subset2, :]
	# The line below will work differently depending on the value of N_subset. If it
	# was set to None, then it will calculate rs based on all pairwise combinations
	# of loci. If N_subset was set to a sensible value (no larger than the smaller of
	# the two callsets), then we calculate rs while setting the subset to be equal to
	# the callset sizes. This means that the function will assign each SNP in call_set1
	# to a SNP in call_set2. That random allocation will be different for each bootstrap
	# replication, but I think that's fine.
	rs = twochrom_r_squared(call_set1, call_set2, N_subset)
	W = Weir_Ne(rs, S, c)
	W_bs = np.array([])
	for i in range(n_bootstrap):
		print(i)
		s = np.random.choice(range(S), subsample_size, replace = False)
		c1 = call_set1[:, s]
		c2 = call_set2[:, s]
		#
		singleton_filter_1 = np.logical_and(np.sum(c1, 1) > 1, np.sum(c1, 1) < (2*c1.shape[1] - 1))
		c1 = c1[singleton_filter_1, :]
		singleton_filter_2 = np.logical_and(np.sum(c2, 1) > 1, np.sum(c2, 1) < (2*c2.shape[1] - 1))
		c2 = c2[singleton_filter_2, :]
		# Remove sites where every sample is heterozygous, since this throws off the calculations,
		# and also these are probably dodgy sites anyway.
		full_het_filter_1 = np.all(c1 == 1, 1)
		c1 = c1[~full_het_filter_1, :]
		full_het_filter_2 = np.all(c2 == 1, 1)
		c2 = c2[~full_het_filter_2, :]
		#
		rs_i = twochrom_r_squared(c1, c2, np.min([c1.shape[0], c2.shape[0]]))
		W_bs = np.append(W_bs, Weir_Ne(rs_i, subsample_size, c))
	# Now get the differences between the bootstrap estimates and the observed value.
	sorted_W_bs = np.sort(W_bs)
	# Work out the indices of the percentiles. Because of floating point issues, we need to dance around a bit here.
	min_perc_index = n_bootstrap * round(1-p, 3) / 2
	max_perc_index = int(np.ceil(n_bootstrap - min_perc_index) - 1)
	min_perc_index = int(min_perc_index)
	# Now get the confidence intervals
	min_perc = sorted_W_bs[min_perc_index]
	max_perc = sorted_W_bs[max_perc_index]
	# Now report the mean and confidence intervals, and then all the boostrapped values
	return(W, min_perc, max_perc, sorted_W_bs)

#num_loci = min([g1.shape[0], g2.shape[0]])
#conf_interval_replicates_largepop_nocrash += [Weir_Ne_replication(g1, g2], num_loci)]
#num_loci = min([g1.shape[0], g2.shape[0]])


In [ ]:
replicates1_post = Weir_Ne_replication(g1_post, g2_post, N_subset = 1000)
replicates1_post

In [ ]:
replicates1_pre = Weir_Ne_replication(g1_pre, g2_pre, N_subset = 1000)
replicates1_pre

In [ ]:
replicates2_post = Weir_Ne_bootstrap_by_sample(g1_post, g2_post)
replicates2_post


In [ ]:
replicates2_pre = Weir_Ne_bootstrap_by_sample(g1_pre, g2_pre)
replicates2_pre

In [19]:
with open('r2_allpop_pre.pickle', 'wb') as f :
	pickle.dump((replicates2_pre), f)
with open('r2_allpop_post.pickle', 'wb') as f :
	pickle.dump((replicates2_post), f)

In [ ]:
# Calculate the 5th percentile, median, and 95th percentile
percentile_5 = np.percentile(rs_pre, 5)
median = np.median(rs_pre)
percentile_95 = np.percentile(rs_pre, 95)

print("5th Percentile:", percentile_5)
print("Median:", median)
print("95th Percentile:", percentile_95)

In [ ]:
stats.mannwhitneyu(rs_pre,rs_post)

In [ ]:
# use a large value to represent infinite Ne
r_post = replicates2_post[3]
r_post[np.isinf(r_post)] = 5000000
r_pre = replicates2_pre[3]
r_pre[np.isinf(r_pre)] = 5000000

In [ ]:
import matplotlib.pyplot as plt
# Log-transform the data
log_r_post = np.log10(r_post)
log_r_pre = np.log10(r_pre)

# Plotting side-by-side histograms
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 2)
plt.hist(log_r_post, bins=30, density=True, alpha=0.7, color='blue', edgecolor='black')
plt.xlabel('log(Ne_post) estimates')
plt.ylabel('Frequency')
plt.title('Npbo Post-LLINEUP trial Ne estimates in Eastern Uganda')

plt.subplot(1, 2, 1)
plt.hist(log_r_pre, bins=30, density=True, alpha=0.7, color='green', edgecolor='black')
plt.xlabel('log(Ne_pre) estimates')
plt.ylabel('Frequency')
plt.title('Npbo Pre-LLINEUP trial Ne estimates in Eastern Uganda')

# Adjust layout for better visualization
plt.tight_layout()
#save
plt.savefig('ne_npbo_east_hist.png')
# Show the plot
plt.show()

